# Combine wetlands from wetlands and mangroves datasets into single shapefile

In [ ]:
import subprocess
import warnings

import geopandas as gpd
import numpy as np
import pandas as pd
import pygeos
import xarray as xr
from rhg_compute_tools import kubernetes as rhgk

from sliiders import settings as sset

warnings.filterwarnings("ignore", message=".*initial implementation of Parquet.*")


PATH_GLOBCOVER_NC = sset.PATH_GLOBCOVER_2009.parent / (
    sset.PATH_GLOBCOVER_2009.stem + ".nc"
)
PATH_GLOBCOVER_SHP = sset.PATH_GLOBCOVER_2009.parent / (
    sset.PATH_GLOBCOVER_2009.stem + ".shp"
)

## 1. Get wetland cover from `GLOBCOVER` as `.nc`

In [ ]:
client, cluster = rhgk.get_micro_cluster()

cluster.scale(8)
cluster

In [ ]:
da = xr.open_rasterio(sset.PATH_GLOBCOVER_2009, chunks={"x": 32400, "y": 27900})

In [ ]:
da = da.persist()

In [ ]:
da = da.squeeze("band").drop("band")

In [ ]:
da = da.isin([160, 170, 180]).persist()

In [ ]:
da = da.to_dataset(name="wetlands")

In [ ]:
da = da.compute()

In [ ]:
da.to_netcdf(PATH_GLOBCOVER_NC)

In [ ]:
client.close()
cluster.close()

## 2. Vectorize
### Run `gdal_polygonize.py` in shell

In [ ]:
cmd = f"gdal_polygonize.py {str(PATH_GLOBCOVER_NC)} {str(PATH_GLOBCOVER_SHP)}"

sp = subprocess.Popen(cmd, shell=True)
rc = sp.wait()

print(sp)

## 3. Clean up shapefile

In [ ]:
gdf = gpd.read_file(PATH_GLOBCOVER_SHP)

In [ ]:
gdf = gdf[gdf["DN"] == 1].drop(columns=["DN"])
gdf = gdf.explode().reset_index(drop=True)

In [ ]:
gdf.to_file(PATH_GLOBCOVER_SHP)

## 4. Combine with mangroves

In [ ]:
globcover_mask = gpd.read_file(PATH_GLOBCOVER_SHP)

In [ ]:
mangroves = gpd.read_file(sset.PATH_GLOBAL_MANGROVES)

In [ ]:
globcover_mask["geometry"] = pygeos.to_shapely(
    pygeos.make_valid(pygeos.from_shapely(globcover_mask["geometry"]))
)

In [ ]:
combined = gpd.sjoin(globcover_mask, mangroves, how="left")

In [ ]:
mangroves = mangroves.reset_index(drop=False)

In [ ]:
mangroves["in_combined"] = mangroves["index"].isin(combined["index_right"].unique())

In [ ]:
globcover_only = combined[combined["index_right"].isnull()].copy()
combined = combined[combined["index_right"].notnull()].copy()

combined = combined.reset_index(drop=True)

In [ ]:
combined["index_right"] = combined["index_right"].astype(int)

In [ ]:
combined["geometry_right"] = gpd.GeoSeries(
    np.take(mangroves["geometry"].to_numpy(), combined["index_right"].to_numpy())
)

In [ ]:
combined = combined.set_geometry("geometry_right")

In [ ]:
combined = combined.dissolve("FID").reset_index(drop=False)

In [ ]:
combined["geometry"] = combined["geometry"].difference(combined["geometry_right"])

In [ ]:
combined = combined.set_geometry("geometry")

In [ ]:
combined = pd.concat([combined, mangroves, globcover_only], ignore_index=True)

In [ ]:
combined = combined[["geometry"]].copy()

In [ ]:
combined.iloc[:20].plot()

In [ ]:
combined = combined.explode().reset_index(drop=True)

In [ ]:
combined = combined[combined.geometry.area > 0].copy()

In [ ]:
combined.to_file(sset.PATH_WETLANDS_INT)